In [35]:
import numpy as np
import pandas
import os, random, pickle
from sklearn.metrics import confusion_matrix
random.seed(0)
np.random.seed(0)

In [36]:
def getdata(feature_file_path,label_file_path, file_name):
    with open(feature_file_path) as f:
        f1_tx = f.readlines()
    
    with open(label_file_path) as f:
        f1_ty = f.read()
    
    a = []

    for i in range(len(f1_tx)):
        d = f1_tx[i]
        b = [float(j) for j in d.split()]
        b = b[:30]
        a.append(np.array(b))

    f1_train_x = np.array(a)
    f1_train_y = np.array([int(i) for i in f1_ty.split()])
    
    with open(file_name+'.pkl', 'wb') as f:
        pickle.dump([f1_train_x,f1_train_y], f)

In [39]:
getdata('data/trPCA_03.txt', 'data/TtrPCA_03.txt', 'f3_train')

In [40]:
getdata('data/tsPCA_03.txt', 'data/TtsPCA_03.txt', 'f3_test')

In [47]:
with open('f3_train.pkl', 'rb') as handle:
    b = pickle.load(handle)
    X,y=b

In [48]:
with open('f3_test.pkl', 'rb') as handle:
    b = pickle.load(handle)
    test_x,test_y=b

In [18]:
class multi_case_1():
    def __init__ (self,mus,sigmas,priors):
        self.mus=mus
        self.sigmas=sigmas
        self.priors=priors
        values=self.calculate_case_1()
        print(f"Paramter Values calculated: ", values)
        return None
    
    def calculate_case_1(self):
        mu_diff=(self.mus[0]-self.mus[1])
        self.w= mu_diff
        self.x0= 1/2 * (self.mus[0]+self.mus[1]) - (self.sigmas[0][0]**2)/((np.linalg.norm(mu_diff))**2)*(math.log(priors[0]/priors[1])*mu_diff)
        return self.w,self.x0
    
    def decision_boundry_fn(self,x):
        return np.matmul(self.w.T, (x-self.x0))
    
    def predict(self,X):
        op_data=[np.matmul(self.w.T, (x-self.x0)) for x in X]
        to_ret=[1 if x>0 else 2 for x in op_data]
        return np.array(to_ret)

In [19]:
class multi_case_2():
    def __init__ (self,mus,sigmas,priors):
        self.mus=mus
        self.sigmas=sigmas
        self.priors=priors
        values=self.calculate_case_2()
        print(f"Paramter Values calculated: ", values)
        return None
    
    def calculate_case_2(self):
        sigma_inv=np.linalg.inv(self.sigmas[0])
        mu_diff=self.mus[0]-self.mus[1]
        self.w=np.dot(sigma_inv,mu_diff)
        self.x0=((self.mus[0]+self.mus[1])/2)-np.log(self.priors[0]/priors[1])*mu_diff/(np.matmul(np.matmul(mu_diff.T,sigma_inv),mu_diff))
        return self.w,self.x0
    
    def decision_boundry_fn(self,x):
        return np.matmul(self.w.T, (x-self.x0))
    
    def predict(self,X):
        op_data=[np.matmul(self.w.T, (x-self.x0)) for x in X]
        to_ret=[1 if x>0 else 2 for x in op_data]
        return np.array(to_ret)

In [26]:
class multi_case_3():
    def __init__ (self,mus,sigmas,priors):
        self.mus=mus
        self.sigmas=sigmas
        self.priors=priors
        values=self.calculate_case_3_constants()
        # print(f"Paramter Values calculated: {self.W12}\n{self.W22}\n{self.w11}\n{self.w21}\n{self.w10}\n{self.w20}")
        
        return None
    
    def calculate_case_3_constants(self):
        dimension=len(self.mus[0])
        # print(self.sigmas.shape)
        if (dimension,dimension) != self.sigmas[0].shape:
            return NameError("Dimension mismatch")
        mu1=self.mus[0]
        mu2=self.mus[1]
        sigma1=self.sigmas[0]
        sigma2=self.sigmas[1]
        sigma1_inv=np.linalg.inv(sigma1)
        sigma2_inv=np.linalg.inv(sigma2)
        prior1=self.priors[0]
        prior2=self.priors[1]
        self.W12=-1/2*sigma1_inv
        self.W22=-1/2*sigma2_inv
        self.w11=np.matmul(sigma1_inv,mu1)
        self.w21=np.matmul(sigma2_inv,mu2)
        self.w10=-1/2*np.matmul(np.matmul(mu1.T,sigma1_inv),mu1) - 1/2*np.log(np.linalg.det(sigma1)) + np.log(prior1)
        self.w20=-1/2*np.matmul(np.matmul(mu2.T,sigma2_inv),mu2) - 1/2*np.log(np.linalg.det(sigma2)) + np.log(prior2)
        return self.W12,self.W22,self.w11,self.w21,self.w10,self.w20
    
    def decision_boundry_fn(self,x):
        return np.matmul(np.matmul(x.T,(self.W12-self.W22)), x) + np.matmul((self.w11.T-self.w21.T),x) + (self.w10-self.w20) 
    
    def predict(self,X):
        op_data=[np.matmul(np.matmul(x.T,(self.W12-self.W22)), x) + np.matmul((self.w11.T-self.w21.T),x) + (self.w10-self.w20) for x in X]
        to_ret=[1 if x>0 else 2 for x in op_data]
        return np.array(to_ret)
    
    def getparams(self):
        return self.W12,self.W22,self.w11,self.w21,self.w10,self.w20
    
    def predict_with_params(self,X,W12,W22,w11,w21,w10,w20):
        op_data=[np.matmul(np.matmul(x.T,(W12-W22)), x) + np.matmul((w11.T-w21.T),x) + (w10-w20) for x in X]
        to_ret=[1 if x>0 else 2 for x in op_data]
        return np.array(to_ret)
        

In [27]:
def calculate_cov(X):
    X=np.array(X)
    N, M = X.shape
    # Calculate the covariance matrix
    cov = np.zeros((M, M))
    for i in range(M):
        # Mean of column "i"
        mean_i = np.sum(X[:, i]) / N
        for j in range(M):
            # Mean of column "j"
            mean_j = np.sum(X[:, j]) / N
            # Covariance between column "i" and column "j"
            cov[i, j] = np.sum((X[:, i] - mean_i) * (X[:, j] - mean_j)) / N
    return cov

In [28]:
def max_likelihood(X):
    mean=np.mean(X,axis=0)
    cov=calculate_cov(X)
    return mean,cov

In [29]:
def misclf_rate(y_true, y_pred):
    y_true=list(y_true)
    y_pred=list(y_pred)
    total_class1 = y_true.count(1)
    total_class2 = y_true.count(2)
    misclf_class1 = 0
    misclf_class2 = 0    
    
    # print(y_pred)
    
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()   
        
    total_misclassification_rate = (fp+fn)/(tn+fp+fn+tp)   
    for i in range(len(y_true)):
        if y_true[i] != y_pred[i]:
            if y_true[i] == 1:
                misclf_class1 += 1
            elif y_true[i] == 2:
                misclf_class2 += 1   
            
    class1_misclf_rate = misclf_class1/total_class1
    class2_misclf_rate = misclf_class2/total_class2    
    return class1_misclf_rate,class2_misclf_rate,total_misclassification_rate

# c1_mis,c2_mis,tot_mis=misclf_rate(y_true,y_pred)
# c1_mis,c2_mis,tot_mis

In [30]:
def error_rate_for(percentage,X,y, test_x, test_y):
    np.random.seed(35)
       
    X0_=np.array([list(X[i]) for i in range(len(y)) if y[i]==1])
    X1_=np.array([list(X[i]) for i in range(len(y)) if y[i]==2])
    # print(int(len(X0_)*0.01*percentage),int(len(X1_)*0.01*percentage))
    
    train_i0=np.random.choice(range(len(X0_)),int(len(X0_)*0.01*percentage),replace=False)
    train_i1=np.random.choice(range(len(X1_)),int(len(X1_)*0.01*percentage),replace=False)
    X0=[X0_[i] for i in train_i0]
    X1=[X1_[i] for i in train_i1]
    
    prior0=0.5 #len(X0)/(len(X0)+len(X1))
    prior1=0.5 #len(X1)/(len(X0)+len(X1))
    
    
    mean0,cov0=max_likelihood(X0)
    mean1,cov1=max_likelihood(X1)
    

    mus=np.array([mean0,mean1])
    sigmas=np.array([cov0,cov1])
    priors=np.array([prior0,prior1])
    # print(f"mus={mus}\n\n,sigmas={sigmas}\n\n,priors={priors}")
    
    if np.array_equal(cov0,cov1):
        if cov0[0][0]==cov0[1][1] and cov0[0][1]==0 and cov0[1][0]==0:
            model=multi_case_1(mus,sigmas,priors)
        else:
            model=multi_case_2(mus,sigmas,priors)
    else:
        model=multi_case_3(mus,sigmas,priors)
        
    W12, W22, w11, w21, w10, w20 = model.getparams() 
    y_test_pred=model.predict_with_params(test_x, W12, W22, w11, w21, w10, w20)
    # y_pred=model.predict(X)
    
    # print("prediction_shape vs y_true shape:",y_pred.shape, "   ", y.shape)
    
    c1_mis,c2_mis,tot_mis=misclf_rate(y_true=test_y,y_pred=y_test_pred)
    
    return c1_mis,c2_mis,tot_mis

In [51]:
c1_mis, c2_mis, tot_mis =error_rate_for(percentage=100,X=X,y=y, test_x= X, test_y= y)

In [52]:
c1_mis, c2_mis, tot_mis

(0.0, 0.0, 0.0)